In [360]:
import pandas as pd

In [361]:
train_df = pd.read_csv('/mnt/Linux/Learning/Python/Kaggle/Predict Calorie Expenditure/train.csv')
train_df.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [362]:
train_df.describe()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253,88.282781
std,216506.495284,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875,62.395349
min,0.000000,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,187499.750000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,374999.500000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,562499.250000,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,749999.000000,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [378]:
test_df = pd.read_csv('/mnt/Linux/Learning/Python/Kaggle/Predict Calorie Expenditure/test.csv')

In [364]:
train_df.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories'],
      dtype='object')

In [365]:
train_df.dtypes

id              int64
Sex            object
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
Calories      float64
dtype: object

In [366]:
from sklearn.preprocessing import LabelEncoder

def feature_engineering(data):
    # Create new features
    data['Age_Height'] = data['Age'] * data['Height']
    data['Weight_Duration'] = data['Weight'] * data['Duration']

    # Convert categorical variables to numerical
    le = LabelEncoder()
    data['Sex'] = le.fit_transform(data['Sex'])

    # Drop unnecessary columns
    data.drop(columns=['id', 'Age', 'Height'], inplace=True)

    return data

In [367]:
feature_engineering(train_df)
train_df.head()

,Sex,Weight,Duration,Heart_Rate,Body_Temp,Calories,Age_Height,Weight_Duration
0,1,82.0,26.0,101.0,41.0,150.0,6804.0,2132.0
1,0,60.0,8.0,85.0,39.7,34.0,10432.0,480.0
2,0,64.0,7.0,84.0,39.8,29.0,8211.0,448.0
3,1,90.0,25.0,105.0,40.7,140.0,3840.0,2250.0
4,0,61.0,25.0,102.0,40.6,146.0,6308.0,1525.0


In [368]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_boxplot_all(data):
    columns = data.columns
    for column in columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=data[column])
        plt.title(f'Boxplot of {column}')
        plt.show()

# plot_boxplot_all(custom_set2)

In [369]:
# def IQR_outlier(data):
#     Q1 = data.quantile(0.25)
#     Q3 = data.quantile(0.75)
#     IQR = Q3 - Q1
#     return data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]
#
# IQR_outlier(train_df)

In [370]:
# plot_boxplot_all(custom_set2)

In [371]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming `custom_set2` is your preprocessed dataset
X = train_df.drop(columns=['Calories'])
y = train_df['Calories']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the scaler on the training set
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test set using the same scaler
X_test_scaled = scaler.transform(X_test)

In [372]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Define models
models = {
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "LinearRegression": LinearRegression()
}

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
predictions = {name: [] for name in models.keys()}
true_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    fold_preds = []
    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        predictions[name].extend(preds)
        fold_preds.append(preds)

    # Store true values
    true_values.extend(y_test)

# Ensemble predictions (e.g., averaging)
ensemble_preds = np.mean([np.array(predictions[name]) for name in models.keys()], axis=0)

# Evaluate ensemble
mse = mean_squared_error(true_values, ensemble_preds)
print(f"Ensemble Mean Squared Error: {mse}")

Ensemble Mean Squared Error: 27.098105810293735


In [334]:
# from sklearn.ensemble import GradientBoostingRegressor
# # Initialize the Gradient Boosting Regressor
# gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.3, max_depth=3, random_state=42)
#
# # Train the model
# gbr.fit(X_train, y_train)
#
# # Predict on the test set
# y_pred = gbr.predict(X_test)

In [335]:
# Evaluate the model
# from sklearn.metrics import root_mean_squared_error
# rmse = root_mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {rmse}")

Mean Squared Error: 4.440178623706272


In [373]:
#find R2 score
from sklearn.metrics import r2_score
r2 = r2_score(true_values, ensemble_preds)
print(f"R2 Score: {r2}")

R2 Score: 0.9930395859265426


In [379]:
test_df

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5
...,...,...,...,...,...,...,...,...
249995,999995,female,56,159.0,62.0,6.0,85.0,39.4
249996,999996,male,32,202.0,101.0,3.0,84.0,38.4
249997,999997,female,31,164.0,64.0,14.0,98.0,40.1
249998,999998,female,62,158.0,61.0,25.0,106.0,40.7


In [380]:
#prefict test_df

test_df = feature_engineering(test_df)
# test_df.drop(columns=['id'], inplace=True)
test_df_scaled = scaler.transform(test_df)

# Predict using the ensemble model
ensemble_test_preds = np.mean([model.predict(test_df_scaled) for model in models.values()], axis=0)
ensemble_test_preds

/home/tsr/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/home/tsr/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/tsr/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([177.18214138, 177.05571758, 179.41935193, ..., 179.71261293,
       179.57591461, 177.5846659 ])

In [344]:
#compare highScore_submission.csv with test_pred
highScore_submission = pd.read_csv('/mnt/Linux/Learning/Python/Kaggle/Predict Calorie Expenditure/hightScore_submission.csv')
highScore_submission.drop(columns=['id'], inplace=True)

In [231]:
submission = pd.read_csv("/mnt/Linux/Learning/Python/Kaggle/Predict Calorie Expenditure/sample_submission.csv")

In [357]:
tsr = pd.DataFrame({
    # 'id': submission['id'],
    'Calories': test_pred
})

In [358]:
tsr = tsr.abs()

In [359]:
from sklearn.metrics import mean_squared_log_error

msle = mean_squared_log_error(highScore_submission, tsr)
print(f"Mean Squared Log Error: {msle}")

Mean Squared Log Error: 0.009590158338929665


In [352]:
tsr.to_csv('/mnt/Linux/Learning/Python/Kaggle/Predict Calorie Expenditure/Submission3_TSR.csv', index=False)

In [348]:
score = r2_score(highScore_submission, tsr)
score

0.9981125522792866